# CacheBackedEmbeddings

Embeddings는 재계산을 피하기 위해 저장되거나 일시적으로 캐시될 수 있습니다.

Embeddings를 캐싱하는 것은 `CacheBackedEmbeddings`를 사용하여 수행될 수 있습니다. 캐시 지원 embedder는 embeddings를 키-값 저장소에 캐싱하는 embedder 주변에 래퍼입니다. 텍스트는 해시되고 이 해시는 캐시에서 키로 사용됩니다.

`CacheBackedEmbeddings`를 초기화하는 주요 지원 방법은 `from_bytes_store`입니다. 이는 다음 매개변수를 받습니다:

- underlying_embedder: 임베딩을 위해 사용되는 embedder.
- document_embedding_cache: 문서 임베딩을 캐싱하기 위한 [`ByteStore`](/docs/integrations/stores/) 중 하나.
- namespace: (선택 사항, 기본값은 `""`) 문서 캐시를 위해 사용되는 네임스페이스. 이 네임스페이스는 다른 캐시와의 충돌을 피하기 위해 사용됩니다. 예를 들어, 사용된 임베딩 모델의 이름으로 설정하십시오.

**주의**: 동일한 텍스트가 다른 임베딩 모델을 사용하여 임베딩될 때 충돌을 피하기 위해 `namespace` 매개변수를 설정하는 것이 중요합니다.


In [1]:
from langchain.embeddings import CacheBackedEmbeddings

## VectorStore 에서 임베딩 사용

먼저, 로컬 파일 시스템을 사용하여 임베딩을 저장하고 FAISS 벡터 스토어를 사용하여 검색하는 예제를 살펴보겠습니다.


In [ ]:
%pip install --upgrade --quiet  langchain-openai faiss-cpu

- `LocalFileStore`, `TextLoader`, `FAISS`, `OpenAIEmbeddings`, 그리고 `CharacterTextSplitter` 모듈을 임포트합니다.
- `OpenAIEmbeddings`를 사용하여 언더라이닝 임베딩을 초기화합니다.
- `LocalFileStore`를 사용하여 로컬 파일 저장소 인스턴스를 `./cache/` 디렉토리에 생성합니다.
- `CacheBackedEmbeddings`를 사용하여 캐시 지원 임베딩을 초기화하며, 이는 `underlying_embeddings`와 로컬 파일 저장소 인스턴스를 사용하고, 네임스페이스로 `underlying_embeddings.model`을 설정합니다.


In [3]:
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding,
    store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [4]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

['version.txt', 'hub/version.txt']

문서를 로드하고, 청크로 분할한 다음, 각 청크를 임베딩하고 벡터 저장소에 로드합니다.


- `TextLoader`를 사용하여 "../../state_of_the_union.txt" 파일에서 텍스트 데이터를 로드합니다.
- `CharacterTextSplitter`를 초기화하여 문서를 1000자 단위로 분할합니다. 이때, 분할된 텍스트 간의 겹침은 없습니다.
- 분할된 문서들은 `documents` 변수에 저장됩니다.


In [7]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("./data/appendix-keywords.txt").load()
# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [9]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: user 218 ms, sys: 26.7 ms, total: 245 ms
Wall time: 951 ms


벡터 저장소를 다시 생성하려고 하면, 임베딩을 다시 계산할 필요가 없기 때문에 훨씬 더 빠르게 처리됩니다.


In [11]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 3.6 ms, sys: 1.25 ms, total: 4.85 ms
Wall time: 4.23 ms


In [13]:
# VectorStore에서 처음 5개의 키를 가져옵니다.
list(store.yield_keys())[:5]

['text-embedding-ada-0020fd71f95-1342-512d-9d5b-3e3ab3c6bbe0',
 'text-embedding-ada-00274ae75af-9058-555e-aefa-082f0b4e0560',
 'text-embedding-ada-00241e7391b-b68f-5e9f-bb07-3609bb83c3e2',
 'text-embedding-ada-0027494a7c8-3399-52a1-85ef-f4d0a563d31f',
 'text-embedding-ada-0029db9e1cd-62d8-50fc-94f4-24bef3cacaf5']

## `ByteStore` 로 변경

다른 `ByteStore`를 사용하기 위해서는 `CacheBackedEmbeddings`를 생성할 때 해당 `ByteStore`를 사용하면 됩니다.

아래에서는, 비영구적인 `InMemoryByteStore`를 사용하여 동일한 캐시된 임베딩 객체를 생성하는 예시를 보여줍니다.


In [15]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)